In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [44]:
batch_size = 256
epoch = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [39]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('dataset', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)

In [23]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)

In [34]:
class ConvNet(nn.Module):
    """
    Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
    dilation=1, groups=1, bias=True, padding_mode='zeros')
    """
    def __init__(self):
        super().__init__()
        # input: 1,28*28
        self.conv1 = nn.Conv2d(1, 10, 5) # 
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20*10*10, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        """x: 1,28*28
        """
        in_size = x.size(0)
        # 1,28*28 -> 10, 24*24 -> 10, 24*24 -> 10, 12*12
        out = F.max_pool2d(F.relu(self.conv1(x)), 2, 2)
        # 10, 12*12 -> 20, 10*10 -> 20, 10*10
        out = F.relu(self.conv2(out))
        
        out = out.view(in_size, -1)
        # 20, 10*10 -> 500 -> 10
        out = self.fc2(F.relu(self.fc1(out)))
        out = F.log_softmax(out, dim=1)
        return out

In [35]:
model = ConvNet().to(device)
optimizer = optim.Adam(model.parameters())

In [36]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [37]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
for epoch in range(1, epoch + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.026057
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.017528
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.045409

Test set: Average loss: 0.0296, Accuracy: 9907/10000 (99%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.013430
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.008060
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.014223

Test set: Average loss: 0.0341, Accuracy: 9890/10000 (99%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.009545
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.021086
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.032012

Test set: Average loss: 0.0291, Accuracy: 9906/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.016935
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.008823
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.018348

Test set: Average loss: 0.0292, Accuracy: 9912/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.004490
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.009725
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.003877

Test 